# Bottom-up Experiments

These are the experiments to test the performance of the bottom-up strategy. If you are running an experiment to test the validity of a particular design space, please make sure to load the approrpiate OFA network before doing so. The cells corresponding to each design space have been marked with "DESIGN SPACE: <NAME>" at the top of the cell.
    
We do not recommend running any of the experiments as they may take a while (we average over 10 runs). Please run the cell under the DEMO section to see bottom-up in action. Before running it, make sure you run all cells in the PREP section

# PREP

In [32]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import numpy as np
import time
import random
import math
import copy
from matplotlib import pyplot as plt

In [33]:
import ofa
import ofa.model_zoo
import ofa.tutorial
from ofa.tutorial import LatencyTableReal
import ofa.nas.accuracy_predictor.arch_encoder
import ofa.nas.search_algorithm
import ofa.nas.efficiency_predictor.latency_lookup_table

In [34]:
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
print('Successfully imported all packages and configured random seed to %d!'%random_seed)

cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    print('Using GPU.')
else:
    print('Using CPU.')

Successfully imported all packages and configured random seed to 1!
Using CPU.


In [35]:
# DESIGN SPACE: MobileNetV3
ofa_network = ofa.model_zoo.ofa_net('ofa_mbv3_d234_e346_k357_w1.2', pretrained=True)
print('The OFA Network is ready.')

The OFA Network is ready.


In [36]:
target_hardware = 'note10'
latency_table = ofa.tutorial.LatencyTable(device=target_hardware)
print('The Latency lookup table on %s is ready!' % target_hardware)

Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/160_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/160_lookup_table.yaml


Built latency table for image size: 160.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/176_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/176_lookup_table.yaml


Built latency table for image size: 176.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/192_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/192_lookup_table.yaml


Built latency table for image size: 192.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/208_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/208_lookup_table.yaml


Built latency table for image size: 208.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/224_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/224_lookup_table.yaml


Built latency table for image size: 224.
The Latency lookup table on note10 is ready!


In [37]:
if cuda_available:
    # path to the ImageNet dataset
    print("Please input the path to the ImageNet dataset.\n")
    imagenet_data_path = input()
    #imagenet_data_path = 'C:\School\once-for-all-master\imgnet'

    # if 'imagenet_data_path' is empty, download a subset of ImageNet containing 2000 images (~250M) for test
    if not os.path.isdir(imagenet_data_path):
        os.makedirs(imagenet_data_path, exist_ok=True)
        ofa.utils.download_url('https://hanlab.mit.edu/files/OnceForAll/ofa_cvpr_tutorial/imagenet_1k.zip', model_dir='data')
        ! cd data && unzip imagenet_1k 1>/dev/null && cd ..
        ! copy -r data/imagenet_1k/* $imagenet_data_path
        ! del -rf data
        print('%s is empty. Download a subset of ImageNet for test.' % imagenet_data_path)

    print('The ImageNet dataset files are ready.')
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.


In [38]:
if cuda_available:
    # The following function build the data transforms for test
    def build_val_transform(size):
        return transforms.Compose([
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])

    data_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(
            root=os.path.join(imagenet_data_path, 'val'),
            transform=build_val_transform(224)
        ),
        batch_size=250,  # test batch size
        shuffle=True,
        num_workers=16,  # number of workers for the data loader
        pin_memory=True,
        drop_last=False,
    )
    print('The ImageNet dataloader is ready.')
else:
    data_loader = None
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.


In [39]:
#accuracy_predictor = ofa.nas.accuracy_predictor.AccuracyPredictor(
accuracy_predictor = ofa.tutorial.AccuracyPredictor(
    pretrained=True,
    device='cuda:0' if cuda_available else 'cpu'
)

print('The accuracy predictor is ready!')

The accuracy predictor is ready!


In [5]:
# DESIGN SPACE: Resnet50D
ofa_network = ofa.model_zoo.ofa_net('ofa_resnet50', pretrained=True)
print('The OFA Network is ready.')

The OFA Network is ready.


In [6]:
# DESIGN SPACE: ProxylessNAS
ofa_network = ofa.model_zoo.ofa_net('ofa_proxyless_d234_e346_k357_w1.3', pretrained=True)
print('The OFA Network is ready.')

The OFA Network is ready.


## Experiments

In [40]:
def run_bottom_up_evolutionary_search(latency_constraint):
#     latency_constraint = (20,25,30)  # ms, suggested range [15, 33] ms
    P = 100  # The size of population in each generation
    N = 500  # How many generations of population to be searched
    N2 = 100
    r = 0.25  # The ratio of networks that are used as parents for next generation
    params = {
        'constraint_type': target_hardware, # Let's do FLOPs-constrained search
        'efficiency_constraint': latency_constraint,
        'mutate_prob': 0.1, # The probability of mutation in evolutionary search
        'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
        'efficiency_predictor': latency_table, # To use a predefined efficiency predictor.
        'accuracy_predictor': accuracy_predictor, # To use a predefined accuracy_predictor predictor.
        'population_size': P,
        'max_time_budget': N,
        'max_time_budget2': N2,
        'parent_ratio': r,
    }

    # build the evolution finder
    finder = ofa.tutorial.EvolutionFinder(**params)

    # start searching
    result_lis = []
    st = time.time()
    best_valids, best_info = finder.run_evolution_search_multi()
    for i in range(len(latency_constraint)):
        result_lis.append(best_info[i])
        print('Found best architecture on %s with latency <= %.2f ms'
              'It achieves %.2f%s predicted accuracy with %.2f ms latency on %s.' %
              (target_hardware, latency_constraint[i], best_info[i][0] * 100, '%', best_info[i][-1], target_hardware))

        # visualize the architecture of the searched sub-net
        _, net_config, latency = best_info[i]
        ofa_network.set_active_subnet(ks=net_config['ks'], d=net_config['d'], e=net_config['e'])
        print('Architecture of the searched sub-net:')
        print(ofa_network.module_str)
    ed = time.time()
    print('Time taken for latency constraint: ', (ed-st))
    return ed-st

## 1. Running Time for k Latency Constraints

### MobileNetV3

In [15]:
latency_constraints = (15, 20, 25, 30, 35)
times = {}
for i in range(len(latency_constraints)):
    latency_constraint = latency_constraints[:i+1]
    temp_times = 0
    for j in range(10):
        temp_times += run_bottom_up_evolutionary_search(latency_constraint)
    times[latency_constraint] /= 10

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:17<00:00, 28.24it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.26% predicted accuracy with 14.92 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E4.0, K3), Identity)
(SE(O48, E3.0, K5), None)
(SE(O48, E4.0, K3), Identity)
((O96, E4.0, K7), None)
((O96, E3.0, K7), Identity)
((O96, E4.0, K3), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E3.0, K5), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K5), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Time taken for latency constraint:  26.16634511947632


Searching with note10 constraint (20): 100%|██████████| 100/100 [00:03<00:00, 26.08it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.64% predicted accuracy with 14.90 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K3), Identity)
((O96, E4.0, K5), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K5), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E4.0, K3), Identity)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.30% predicted accuracy with 19.72 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O

Searching with note10 constraint (25): 100%|██████████| 100/100 [00:03<00:00, 26.22it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.52% predicted accuracy with 14.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E3.0, K3), Identity)
(SE(O48, E3.0, K5), Identity)
((O96, E6.0, K5), None)
((O96, E4.0, K3), Identity)
((O96, E3.0, K5), Identity)
(SE(O136, E4.0, K5), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E3.0, K5), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.33% predicted accuracy with 19.96 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E3.0, K3), Identity)
(SE(O4

Searching with note10 constraint (30): 100%|██████████| 100/100 [00:04<00:00, 24.00it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.49% predicted accuracy with 14.90 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E3.0, K3), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K5), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E4.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.31% predicted accuracy with 19.97 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K3), Identity)
(SE(O136, E4

Searching with note10 constraint (35): 100%|██████████| 100/100 [00:04<00:00, 24.32it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.67% predicted accuracy with 14.88 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O48, E3.0, K3), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K3), Identity)
((O96, E4.0, K5), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E4.0, K5), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E4.0, K3), Identity)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.29% predicted accuracy with 19.85 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E4.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O

In [16]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
temp_times = 0
for j in range(10):
    temp_times += run_bottom_up_evolutionary_search(latency_constraints)
times[latency_constraints] /= 10

Searching with note10 constraint (60): 100%|██████████| 100/100 [00:03<00:00, 28.81it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.51% predicted accuracy with 14.92 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E4.0, K3), Identity)
((O96, E4.0, K7), None)
((O96, E4.0, K5), Identity)
(SE(O136, E4.0, K7), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E3.0, K3), Identity)
(SE(O192, E4.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.56% predicted accuracy with 20.00 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E6.0, K3), None)
(SE(O48, E6.0, K3), Identity)
(SE

In [17]:
times

{(15,): 26.16634511947632,
 (15, 20): 31.591012001037598,
 (15, 20, 25): 38.2353081703186,
 (15, 20, 25, 30): 43.80901288986206,
 (15, 20, 25, 30, 35): 47.78053402900696,
 (15, 20, 25, 30, 35, 40, 45, 50, 55, 60): 64.29084706306458}

### ResNet50D

In [19]:
latency_constraints = (15, 20, 25, 30, 35)
times = {}
for i in range(len(latency_constraints)):
    latency_constraint = latency_constraints[:i+1]
    times[latency_constraint] = run_bottom_up_evolutionary_search(latency_constraint)

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:21<00:00, 23.12it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.63% predicted accuracy with 14.88 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpoo

Searching with note10 constraint (20): 100%|██████████| 100/100 [00:04<00:00, 21.14it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.68% predicted accuracy with 14.96 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpool

Searching with note10 constraint (25): 100%|██████████| 100/100 [00:03<00:00, 25.51it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.54% predicted accuracy with 14.97 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->2048_S2, avgpool

Searching with note10 constraint (30): 100%|██████████| 100/100 [00:03<00:00, 27.19it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.48% predicted accuracy with 14.94 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->2048_S2, avgpool

Searching with note10 constraint (35): 100%|██████████| 100/100 [00:03<00:00, 27.10it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.48% predicted accuracy with 14.94 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpoo

In [20]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
times[latency_constraints] = run_bottom_up_evolutionary_search(latency_constraints)

Searching with note10 constraint (60): 100%|██████████| 100/100 [00:03<00:00, 30.86it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.43% predicted accuracy with 14.93 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->2048_S2, avgpoo

In [21]:
times

{(15,): 30.703929901123047,
 (15, 20): 37.73849034309387,
 (15, 20, 25): 38.1832230091095,
 (15, 20, 25, 30): 43.44956994056702,
 (15, 20, 25, 30, 35): 47.836430072784424,
 (15, 20, 25, 30, 35, 40, 45, 50, 55, 60): 63.43661594390869}

### ProxylessNAS

In [23]:
latency_constraints = (15, 20, 25, 30, 35)
times = {}
for i in range(len(latency_constraints)):
    latency_constraint = latency_constraints[:i+1]
    times[latency_constraint] = run_bottom_up_evolutionary_search(latency_constraint)

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:20<00:00, 23.83it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.73% predicted accuracy with 14.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E4.0, K3), Identity)
((O104, E4.0, K5), None)
((O104, E4.0, K3), Identity)
((O104, E3.0, K5), Identity)
((O104, E3.0, K3), Identity)
((O128, E4.0, K3), None)
((O128, E4.0, K3), Identity)
((O128, E3.0, K5), Identity)
((O248, E4.0, K3), None)
((O248, E6.0, K3), Identity)
((O248, E4.0, K7), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Time taken for latency constraint:  29.050503969192505


Searching with note10 constraint (20): 100%|██████████| 100/100 [00:04<00:00, 22.99it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.48% predicted accuracy with 14.94 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E4.0, K3), Identity)
((O56, E3.0, K3), None)
((O56, E4.0, K3), Identity)
((O56, E3.0, K3), Identity)
((O104, E6.0, K3), None)
((O104, E3.0, K5), Identity)
((O104, E4.0, K5), Identity)
((O128, E4.0, K5), None)
((O128, E4.0, K3), Identity)
((O128, E3.0, K3), Identity)
((O248, E4.0, K3), None)
((O248, E4.0, K3), Identity)
((O248, E4.0, K7), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.41% predicted accuracy with 19.97 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E3.0, K3), Identity)
((O5

Searching with note10 constraint (25): 100%|██████████| 100/100 [00:04<00:00, 24.92it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.55% predicted accuracy with 14.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O56, E4.0, K3), Identity)
((O56, E3.0, K5), Identity)
((O104, E6.0, K5), None)
((O104, E4.0, K7), Identity)
((O104, E3.0, K5), Identity)
((O128, E4.0, K3), None)
((O128, E4.0, K3), Identity)
((O128, E3.0, K5), Identity)
((O248, E4.0, K3), None)
((O248, E6.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.25% predicted accuracy with 19.96 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E3.0, K3), Identity)
((O56, E3.0, K5), Identity)
((O10

Searching with note10 constraint (30): 100%|██████████| 100/100 [00:05<00:00, 18.04it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.63% predicted accuracy with 14.96 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O56, E4.0, K3), Identity)
((O104, E6.0, K5), None)
((O104, E4.0, K3), Identity)
((O104, E3.0, K7), Identity)
((O104, E3.0, K3), Identity)
((O128, E3.0, K3), None)
((O128, E3.0, K5), Identity)
((O128, E4.0, K5), Identity)
((O248, E6.0, K3), None)
((O248, E3.0, K7), Identity)
((O248, E3.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.51% predicted accuracy with 19.74 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O56, E4.0, K3), Identity)
((O

Searching with note10 constraint (35): 100%|██████████| 100/100 [00:03<00:00, 25.25it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.75% predicted accuracy with 14.94 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E3.0, K3), Identity)
((O56, E3.0, K5), Identity)
((O104, E3.0, K5), None)
((O104, E4.0, K3), Identity)
((O104, E3.0, K5), Identity)
((O104, E3.0, K5), Identity)
((O128, E4.0, K3), None)
((O128, E4.0, K3), Identity)
((O128, E3.0, K5), Identity)
((O248, E4.0, K3), None)
((O248, E4.0, K3), Identity)
((O248, E4.0, K7), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.30% predicted accuracy with 20.00 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O

In [24]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
times[latency_constraints] = run_bottom_up_evolutionary_search(latency_constraints)

Searching with note10 constraint (60): 100%|██████████| 100/100 [00:03<00:00, 25.70it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.51% predicted accuracy with 14.97 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O56, E4.0, K3), Identity)
((O56, E4.0, K3), Identity)
((O104, E4.0, K7), None)
((O104, E4.0, K3), Identity)
((O104, E3.0, K7), Identity)
((O128, E3.0, K3), None)
((O128, E4.0, K3), Identity)
((O128, E4.0, K3), Identity)
((O248, E6.0, K3), None)
((O248, E3.0, K3), Identity)
((O248, E3.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.43% predicted accuracy with 19.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O56, E4.0, K3), Identity)
((O5

In [25]:
times

{(15,): 29.050503969192505,
 (15, 20): 35.075645208358765,
 (15, 20, 25): 40.246270179748535,
 (15, 20, 25, 30): 52.753483057022095,
 (15, 20, 25, 30, 35): 56.32675790786743,
 (15, 20, 25, 30, 35, 40, 45, 50, 55, 60): 76.1533989906311}

## 2. Accuracy of Discovered Subnetworks

### MobileNetV3

In [27]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
run_bottom_up_evolutionary_search(latency_constraints)

Searching with note10 constraint (60): 100%|██████████| 100/100 [00:03<00:00, 28.73it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.63% predicted accuracy with 14.88 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K3), Identity)
((O96, E6.0, K5), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E3.0, K3), Identity)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.70% predicted accuracy with 19.92 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O

62.214707136154175

### ResNet50D

In [29]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
run_bottom_up_evolutionary_search(latency_constraints)

Searching with note10 constraint (60): 100%|██████████| 100/100 [00:03<00:00, 29.47it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.50% predicted accuracy with 14.94 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpool

63.87831711769104

### ProxylessNAS

In [31]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
run_bottom_up_evolutionary_search(latency_constraints)

Searching with note10 constraint (60): 100%|██████████| 100/100 [00:03<00:00, 28.55it/s]

Found best architecture on note10 with latency <= 15.00 msIt achieves 78.68% predicted accuracy with 14.96 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K5), None)
((O56, E4.0, K3), Identity)
((O56, E3.0, K3), Identity)
((O104, E4.0, K5), None)
((O104, E4.0, K3), Identity)
((O104, E3.0, K5), Identity)
((O128, E4.0, K3), None)
((O128, E4.0, K3), Identity)
((O128, E4.0, K5), Identity)
((O248, E4.0, K3), None)
((O248, E4.0, K3), Identity)
((O248, E4.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.64% predicted accuracy with 19.97 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E3.0, K3), Identity)
((O5

63.48898005485535

## Cost of finding a single latency constraint subnetwork

In [41]:
times = {}
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)

In [48]:
for i in range(len(latency_constraints)):
    lc = latency_constraints[i:i+1]
    times[lc] = run_bottom_up_evolutionary_search(lc)


Searching with note10 constraint (15): 100%|██████████| 500/500 [00:23<00:00, 21.41it/s]


Found best architecture on note10 with latency <= 15.00 msIt achieves 78.47% predicted accuracy with 14.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E4.0, K3), Identity)
(SE(O48, E3.0, K3), None)
(SE(O48, E3.0, K5), Identity)
((O96, E6.0, K5), None)
((O96, E3.0, K3), Identity)
((O96, E4.0, K5), Identity)
((O96, E4.0, K3), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E3.0, K3), Identity)
(SE(O192, E3.0, K3), None)
(SE(O192, E4.0, K3), Identity)
(SE(O192, E4.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Time taken for latency constraint:  31.404531002044678



Searching with note10 constraint (20):  92%|█████████▏| 460/500 [00:23<00:01, 20.95it/s]


Searching with note10 constraint (20): 100%|██████████| 500/500 [00:25<00:00, 19.82it/s]

Searching with note10 constraint (25):   0%|          | 2/500 [00:00<00:25, 19.29it/s]

Found best architecture on note10 with latency <= 20.00 msIt achieves 80.56% predicted accuracy with 19.95 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E4.0, K3), Identity)
((O96, E3.0, K7), None)
((O96, E4.0, K5), Identity)
((O96, E4.0, K3), Identity)
(SE(O136, E4.0, K5), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E4.0, K7), Identity)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Time taken for latency constraint:  25.56589913368225


...

Found best architecture on note10 with latency <= 50.00 msIt achieves 84.35% predicted accuracy with 49.81 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K5), None)
((O32, E4.0, K5), Identity)
((O32, E6.0, K3), Identity)
(SE(O48, E4.0, K7), None)
(SE(O48, E6.0, K3), Identity)
(SE(O48, E4.0, K5), Identity)
(SE(O48, E4.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E3.0, K3), Identity)
((O96, E4.0, K3), Identity)
((O96, E6.0, K3), Identity)
(SE(O136, E3.0, K7), None)
(SE(O136, E6.0, K7), Identity)
(SE(O136, E6.0, K5), Identity)
(SE(O136, E6.0, K3), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E4.0, K5), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Time taken for latency constraint:  19.848477125167847



Searching with note10 constraint (55): 100%|██████████| 500/500 [00:19<00:00, 25.69it/s]

Searching with note10 constraint (60):   1%|          | 3/500 [00:00<00:16, 29.46it/s]

Found best architecture on note10 with latency <= 55.00 msIt achieves 84.52% predicted accuracy with 54.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E6.0, K3), None)
((O32, E3.0, K7), Identity)
((O32, E4.0, K5), Identity)
((O32, E3.0, K5), Identity)
(SE(O48, E6.0, K7), None)
(SE(O48, E4.0, K5), Identity)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E4.0, K5), Identity)
((O96, E3.0, K7), None)
((O96, E3.0, K5), Identity)
((O96, E3.0, K7), Identity)
((O96, E6.0, K5), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E6.0, K5), Identity)
(SE(O136, E6.0, K3), Identity)
(SE(O136, E4.0, K5), Identity)
(SE(O192, E6.0, K5), None)
(SE(O192, E6.0, K7), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E4.0, K7), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Time taken for latency constraint:  19.496007204055786



Searching with note10 constraint (60): 100%|██████████| 500/500 [00:18<00:00, 26.86it/s]

Found best architecture on note10 with latency <= 60.00 msIt achieves 84.74% predicted accuracy with 55.76 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E6.0, K5), None)
((O32, E3.0, K3), Identity)
((O32, E3.0, K3), Identity)
((O32, E6.0, K3), Identity)
(SE(O48, E6.0, K5), None)
(SE(O48, E6.0, K7), Identity)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E6.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K7), Identity)
((O96, E4.0, K7), Identity)
((O96, E6.0, K7), Identity)
(SE(O136, E4.0, K7), None)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E6.0, K5), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E3.0, K5), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Time taken for latency constraint:  18.64280915260315


In [50]:
times.values()

dict_values([31.404531002044678, 25.56589913368225, 22.32000708580017, 20.48673677444458, 21.243536949157715, 26.576290130615234, 20.8565890789032, 19.848477125167847, 19.496007204055786, 18.64280915260315])

# DEMO

Run the cell below to see the bottom up strategy in action!

In [ ]:
latency_constraints = (15, 20, 25, 30, 35)
run_bottom_up_evolutionary_search(latency_constraints)